In [5]:
%matplotlib inline
!pip install scikit-learn
!pip install seaborn
!pip install geopandas matplotlib

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from glob import glob
import geopandas as gpd

from scipy.spatial import distance_matrix
from sklearn.preprocessing import MinMaxScaler

# Para no mostrar 'Future Warnings' producto de que algunos paquetes aún
# trabajan con versiones no actualizadas de otros paquetes
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [6]:
DATOS_generales = pd.read_csv('./datos_proyecto_general.csv')
DATOS_productos = pd.read_csv('./datos_proyecto_productos.csv')

In [7]:
DATOS_generales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5075 entries, 0 to 5074
Data columns (total 36 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Fecha de venta                                    5075 non-null   object 
 1   N.º de pedido                                     5075 non-null   int64  
 2   N.º de comprador                                  4390 non-null   object 
 3   Nombre completo                                   5075 non-null   object 
 4   Nombre                                            5075 non-null   object 
 5   Apellidos                                         4895 non-null   object 
 6   Cantidad de artículos                             5075 non-null   int64  
 7   Forma de pago                                     5075 non-null   object 
 8   Fecha de envío                                    5075 non-null   object 
 9   Calle              

In [8]:
DATOS_productos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6373 entries, 0 to 6372
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Sale Date          6373 non-null   object 
 1   Item Name          6372 non-null   object 
 2   Buyer              5559 non-null   object 
 3   Quantity           6373 non-null   int64  
 4   Price              6373 non-null   float64
 5   Coupon Code        541 non-null    object 
 6   Coupon Details     541 non-null    object 
 7   Discount Amount    6373 non-null   float64
 8   Shipping Discount  6373 non-null   float64
 9   Order Shipping     6373 non-null   float64
 10  Order Sales Tax    6373 non-null   float64
 11  Item Total         6373 non-null   float64
 12  Currency           6373 non-null   object 
 13  Transaction ID     6373 non-null   int64  
 14  Listing ID         6373 non-null   int64  
 15  Date Paid          6373 non-null   object 
 16  Date Shipped       6373 

In [9]:
DATOS_generales_filtrada = DATOS_generales[['Fecha de venta','Nombre','Ciudad','País','Importe del pedido','Cantidad de descuento','Total del pedido']]
DATOS_generales_filtrada.loc[:, 'Fecha de venta'] = pd.to_datetime(DATOS_generales_filtrada['Fecha de venta'], format='%m/%d/%y')
DATOS_generales_filtrada.loc[:, 'Importe del pedido'] = DATOS_generales_filtrada['Importe del pedido'].str.replace(',', '.').astype(float)
DATOS_generales_filtrada.loc[:, 'Cantidad de descuento'] = DATOS_generales_filtrada['Cantidad de descuento'].str.replace(',', '.').astype(float)
DATOS_generales_filtrada.loc[:, 'Total del pedido'] = DATOS_generales_filtrada['Total del pedido'].str.replace(',', '.').astype(float)

DATOS_generales_filtrada.dtypes

C:\Users\Carlosim\AppData\Local\Temp\ipykernel_11580\354603371.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DATOS_generales_filtrada.loc[:, 'Fecha de venta'] = pd.to_datetime(DATOS_generales_filtrada['Fecha de venta'], format='%m/%d/%y')
C:\Users\Carlosim\AppData\Local\Temp\ipykernel_11580\354603371.py:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  DATOS_generales_filtrada.loc[:, 'Fecha de venta'] = pd.to_datetime(DATOS_generales_filtrada['Fecha de venta'], format='%m/%d/%y')
C:\Users\Carlosim\AppData\Local\Temp\ip

Fecha de venta           datetime64[ns]
Nombre                           object
Ciudad                           object
País                             object
Importe del pedido              float64
Cantidad de descuento           float64
Total del pedido                float64
dtype: object

In [10]:
DATOS_productos_filtrada = DATOS_productos[['Sale Date','Item Name','Buyer','Quantity','Price','Discount Amount','Ship City','Ship Country']]
DATOS_productos_filtrada['Sale Date'] = pd.to_datetime(DATOS_productos_filtrada['Sale Date'], format='%m/%d/%y')
DATOS_productos_filtrada.dtypes

C:\Users\Carlosim\AppData\Local\Temp\ipykernel_11580\1668998089.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DATOS_productos_filtrada['Sale Date'] = pd.to_datetime(DATOS_productos_filtrada['Sale Date'], format='%m/%d/%y')


Sale Date          datetime64[ns]
Item Name                  object
Buyer                      object
Quantity                    int64
Price                     float64
Discount Amount           float64
Ship City                  object
Ship Country               object
dtype: object

In [11]:
DATOS_generales_filtrada_ordenada = DATOS_generales_filtrada.sort_values(by='Fecha de venta').reset_index(drop=True)
DATOS_productos_filtrada_ordenada = DATOS_productos_filtrada.sort_values(by='Sale Date').reset_index(drop=True)

In [12]:
DATOS_generales_filtrada_ordenada.dtypes

Fecha de venta           datetime64[ns]
Nombre                           object
Ciudad                           object
País                             object
Importe del pedido              float64
Cantidad de descuento           float64
Total del pedido                float64
dtype: object

In [13]:
DATOS_productos_filtrada_ordenada.dtypes

Sale Date          datetime64[ns]
Item Name                  object
Buyer                      object
Quantity                    int64
Price                     float64
Discount Amount           float64
Ship City                  object
Ship Country               object
dtype: object